In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

coding_poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a natural poet dedicated to writing Haikus about programming languages. Your poetry gives clear, simple, yet profound view of the programming language."
        ),
        (
            "human",
            "Write a Haiku about {language}."
        )
    ]
)

poet_chain = coding_poet_prompt | chat


In [19]:
critic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a critic specialized in explaining Haikus. Your explanations are insightful and easy to understand."
        ),
        (
            "human",
            "\n\nReview the following Haiku and explain its meaning:\n\n{haiku}\n\n"
        )
    ]
)

critic_chain = critic_prompt | chat

final_chain = {"haiku": poet_chain} | critic_chain

final_chain.invoke({"language": "Python"})

Python slithers smooth,
Indentation guides its code,
Elegant and clean.This Haiku beautifully captures the essence of Python programming language. The first line "Python slithers smooth" evokes the sleek and efficient nature of Python, likening it to a snake moving effortlessly. The second line "Indentation guides its code" highlights one of Python's unique features - the use of indentation to structure code, emphasizing the importance of clean and organized formatting. Finally, the last line "Elegant and clean" sums up the overall aesthetic and simplicity of Python code. Overall, this Haiku celebrates Python's elegance, efficiency, and clarity in programming.

AIMessage(content='This Haiku beautifully captures the essence of Python programming language. The first line "Python slithers smooth" evokes the sleek and efficient nature of Python, likening it to a snake moving effortlessly. The second line "Indentation guides its code" highlights one of Python\'s unique features - the use of indentation to structure code, emphasizing the importance of clean and organized formatting. Finally, the last line "Elegant and clean" sums up the overall aesthetic and simplicity of Python code. Overall, this Haiku celebrates Python\'s elegance, efficiency, and clarity in programming.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='lc_run--5062be7d-8c15-4752-9535-f1cbf2312726')